# Turn deck lists into useful format

In [115]:
import requests
import json

#### Scrape some deck lists to work with

In [3]:
deck_lists = scrape_deck_lists()

Requesting the front pages
	Requesting front page number 0
Getting all event ids
	Getting event ids from a front page
Getting deck lists
		Event request for event id 18141
		Event request for event id 18048
		Event request for event id 18142
		Event request for event id 18129
		Event request for event id 18128
		Event request for event id 18060
		Event request for event id 18059
		Event request for event id 18072
		Event request for event id 18074
		Event request for event id 18143
		Deck request for deck id 312465
		Deck request for deck id 312461
		Deck request for deck id 312454
		Deck request for deck id 312389
		Deck request for deck id 312031
		Deck request for deck id 312141
		Deck request for deck id 312395
		Deck request for deck id 312394
		Deck request for deck id 312482
		Deck request for deck id 312144
		Deck request for deck id 312391
		Deck request for deck id 312021
		Deck request for deck id 312393
		Deck request for deck id 312035
		Deck request for deck id 312034
		D

In [18]:
deck_lists.keys()

TypeError: 'dict_keys' object does not support indexing

In [19]:
sample_deck_list = deck_lists[list(deck_lists.keys())[0]]

#### Get all modern legal cards (i think all)

In [116]:
total_pages = 0
url = 'https://api.scryfall.com/cards/search?q=format:modern'
modern_legal_cards_scryfall = []
while True:
    response = requests.get(url, headers={'User-Agent': 'Getting modern legal cards'})
    
    if response.status_code != 200:
        print('Bad Status code on page #{}: {}'.format(response.status_code))
        break
        
    cards = json.loads(response.text)
    
    if not cards['has_more']:
        print('All done! {} pages recieved.'.format(total_pages))
        break
    
    modern_legal_cards_scryfall.append(cards['data'])
    total_pages += 1
    url = cards['next_page']
    
    time.sleep(0.05 + random.random())
    
    print('Page request #{} successful! Onto the next.'.format(total_pages))

Page request #1 successful! Onto the next.
Page request #2 successful! Onto the next.
Page request #3 successful! Onto the next.
Page request #4 successful! Onto the next.
Page request #5 successful! Onto the next.
Page request #6 successful! Onto the next.
Page request #7 successful! Onto the next.
Page request #8 successful! Onto the next.
Page request #9 successful! Onto the next.
Page request #10 successful! Onto the next.
Page request #11 successful! Onto the next.
Page request #12 successful! Onto the next.
Page request #13 successful! Onto the next.
Page request #14 successful! Onto the next.
Page request #15 successful! Onto the next.
Page request #16 successful! Onto the next.
Page request #17 successful! Onto the next.
Page request #18 successful! Onto the next.
Page request #19 successful! Onto the next.
Page request #20 successful! Onto the next.
Page request #21 successful! Onto the next.
Page request #22 successful! Onto the next.
Page request #23 successful! Onto the nex

In [117]:
card_dict = {}
for page in modern_legal_cards_scryfall:
    for card in page:
        # 'id' is scryfalls unique ID
        card_dict[card['name']] = card['id']

There should be 11,348 cards in card_dict (as of 1/15/18) according to scryfall.

    Although, I only get 11,200 as of 1/15/18

In [118]:
len(card_dict)

11200

In [119]:
card_dict['Kumena, Tyrant of Orazca']

'a3aef818-c896-46e6-aaff-56aee52a066c'

In [120]:
sample_deck_list

['4 Tectonic Edge',
 '2 Shefet Dunes',
 '8 Plains',
 '1 Mutavault',
 '4 Horizon Canopy',
 '4 Ghost Quarter',
 '1 Flagstones of Trokair',
 '4 Aether Vial',
 '4 Path to Exile',
 '4 Thraben Inspector',
 '4 Thalia, Guardian of Thraben',
 '2 Serra Avenger',
 '2 Selfless Spirit',
 '3 Restoration Angel',
 '4 Leonin Arbiter',
 '4 Flickerwisp',
 '4 Blade Splicer',
 '1 Aven Mindcensor',
 'Sideboard',
 '1 Sunlance',
 '2 Stony Silence',
 '1 Settle the Wreckage',
 '1 Rest in Peace',
 '1 Relic of Progenitus',
 '2 Oblivion Ring',
 '2 Mirran Crusader',
 "1 Grafdigger's Cage",
 '1 Eidolon of Rhetoric',
 '1 Burrenton Forge-Tender',
 '1 Selfless Spirit',
 '1 Aven Mindcensor',
 '']

In [121]:
card_nums = []
card_names = []
for row in sample_deck_list:
    if row[0] == 'S':
        break
    card_nums.append(int(row[0]))
    card_names.append(row[1:])

In [122]:
card_nums

[4, 2, 8, 1, 4, 4, 1, 4, 4, 4, 4, 2, 2, 3, 4, 4, 4, 1]

In [123]:
card_names

[' Tectonic Edge',
 ' Shefet Dunes',
 ' Plains',
 ' Mutavault',
 ' Horizon Canopy',
 ' Ghost Quarter',
 ' Flagstones of Trokair',
 ' Aether Vial',
 ' Path to Exile',
 ' Thraben Inspector',
 ' Thalia, Guardian of Thraben',
 ' Serra Avenger',
 ' Selfless Spirit',
 ' Restoration Angel',
 ' Leonin Arbiter',
 ' Flickerwisp',
 ' Blade Splicer',
 ' Aven Mindcensor']

In [124]:
temp = {}
user_card_counts = []
for deck_id, deck_list in deck_lists.items():
    card_counts = []
    card_names = []
    for row in sample_deck_list:
        # not worrying about sideboards
        if row[0] == 'S':
            break
        card_count = int(row[0])
        card_name = row[2:]
        user_card_counts.append((int(deck_id), card_name, card_count))
        
        
    temp[int(deck_id)] = {'card_counts': card_counts, 'card_names': card_names}

In [125]:
len(user_card_counts)

1224

In [126]:
with open('user_card_counts.json', 'w') as f:
    json.dump(user_card_counts, f)

In [127]:
import boto3

In [128]:
s3 = boto3.resource('s3')
s3_object = s3.Object('mtg-capstone', 'user_card_count.json')

In [129]:
s3_object.upload_file('user_card_counts.json')

In [77]:
import deck_scraping
import card_scraping
import make_user_card_counts

In [114]:
deck_lists = deck_scraping.scrape_deck_lists(verbose=False)

In [103]:
user_card_counts = make_user_card_counts.make_user_card_counts(deck_lists)

user_card_counts

[(312465, 'Tectonic Edge', 4),
 (312465, 'Shefet Dunes', 2),
 (312465, 'Plains', 8),
 (312465, 'Mutavault', 1),
 (312465, 'Horizon Canopy', 4),
 (312465, 'Ghost Quarter', 4),
 (312465, 'Flagstones of Trokair', 1),
 (312465, 'Aether Vial', 4),
 (312465, 'Path to Exile', 4),
 (312465, 'Thraben Inspector', 4),
 (312465, 'Thalia, Guardian of Thraben', 4),
 (312465, 'Serra Avenger', 2),
 (312465, 'Selfless Spirit', 2),
 (312465, 'Restoration Angel', 3),
 (312465, 'Leonin Arbiter', 4),
 (312465, 'Flickerwisp', 4),
 (312465, 'Blade Splicer', 4),
 (312465, 'Aven Mindcensor', 1),
 (312461, 'Thalia, Guardian of Thraben', 3),
 (312461, 'Unclaimed Territory', 4),
 (312461, 'Godless Shrine', 1),
 (312461, "Avacyn's Pilgrim", 3),
 (312461, 'Plains', 1),
 (312461, 'Noble Hierarch', 4),
 (312461, 'Reflector Mage', 4),
 (312461, "Thalia's Lieutenant", 4),
 (312461, 'Windswept Heath', 4),
 (312461, 'Temple Garden', 1),
 (312461, 'Meddling Mage', 3),
 (312461, 'Forest', 1),
 (312461, 'Kitesail Freebooter

In [109]:
with open('data/user_card_counts2.json', 'w') as f:
    json.dump(user_card_counts, f)

In [130]:
s3 = boto3.client('s3')

In [112]:
filename = 'data/user_card_counts2.json'
bucketname = 'mtg-capstone'


In [113]:
s3.upload_file(filename, bucketname, filename)

In [131]:
deck_list

['1 Become Immense',
 '2 Goblin Lore',
 '4 Faithless Looting',
 '1 Hooting Mandrills',
 '2 Grim Lavamancer',
 '2 Reckless Bushwhacker',
 '3 Flamewake Phoenix',
 '4 Flameblade Adept',
 '4 Burning-Tree Emissary',
 '4 Hollow One',
 '4 Vengevine',
 '4 Insolent Neonate',
 '4 Street Wraith',
 '4 Bomat Courier',
 '1 Forest',
 '3 Mountain',
 '3 Stomping Ground',
 '3 Copperline Gorge',
 '3 Bloodstained Mire',
 '4 Wooded Foothills',
 'Sideboard',
 "3 Tormod's Crypt",
 '2 Ancient Grudge',
 '2 Blood Moon',
 '2 Destructive Revelry',
 '2 Heroic Intervention',
 '4 Lightning Bolt',
 '']

In [132]:
deck_data = '\n'.join(deck_list).encode()

In [134]:
s3.put_object(Bucket='mtg-capstone', Key='data/deck_list_test', Body=deck_data)

{'ETag': '"6af2c03e87d032384adf60228a183b36"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Tue, 16 Jan 2018 19:32:41 GMT',
   'etag': '"6af2c03e87d032384adf60228a183b36"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'mi2p+erHCZ7/iAV3OWxoRczELEJGGynPR3oVmm4Yf/r0IJ01969N+ZXOBA8eaXbHRY7OZA5iXNo=',
   'x-amz-request-id': '524FE991FF8B94BD'},
  'HTTPStatusCode': 200,
  'HostId': 'mi2p+erHCZ7/iAV3OWxoRczELEJGGynPR3oVmm4Yf/r0IJ01969N+ZXOBA8eaXbHRY7OZA5iXNo=',
  'RequestId': '524FE991FF8B94BD',
  'RetryAttempts': 0}}

In [133]:
deck_data

b"1 Become Immense\n2 Goblin Lore\n4 Faithless Looting\n1 Hooting Mandrills\n2 Grim Lavamancer\n2 Reckless Bushwhacker\n3 Flamewake Phoenix\n4 Flameblade Adept\n4 Burning-Tree Emissary\n4 Hollow One\n4 Vengevine\n4 Insolent Neonate\n4 Street Wraith\n4 Bomat Courier\n1 Forest\n3 Mountain\n3 Stomping Ground\n3 Copperline Gorge\n3 Bloodstained Mire\n4 Wooded Foothills\nSideboard\n3 Tormod's Crypt\n2 Ancient Grudge\n2 Blood Moon\n2 Destructive Revelry\n2 Heroic Intervention\n4 Lightning Bolt\n"

In [135]:
s3.list_objects(Bucket='mtg-capstone')

{'Contents': [{'ETag': '"6af2c03e87d032384adf60228a183b36"',
   'Key': 'data/deck_list_test',
   'LastModified': datetime.datetime(2018, 1, 16, 19, 32, 41, tzinfo=tzutc()),
   'Owner': {'DisplayName': 'bwalzer',
    'ID': '2c909b1bc0b8921f585ff633168b033aeb92610e321db68f349d30cc5ef81251'},
   'Size': 465,
   'StorageClass': 'STANDARD'},
  {'ETag': '"cfce016e85c7a02bfa87059934a3cc9e"',
   'Key': 'user_card_count.json',
   'LastModified': datetime.datetime(2018, 1, 16, 19, 12, 32, tzinfo=tzutc()),
   'Owner': {'DisplayName': 'bwalzer',
    'ID': '2c909b1bc0b8921f585ff633168b033aeb92610e321db68f349d30cc5ef81251'},
   'Size': 38080,
   'StorageClass': 'STANDARD'}],
 'EncodingType': 'url',
 'IsTruncated': False,
 'Marker': '',
 'MaxKeys': 1000,
 'Name': 'mtg-capstone',
 'Prefix': '',
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'application/xml',
   'date': 'Tue, 16 Jan 2018 19:33:08 GMT',
   'server': 'AmazonS3',
   'transfer-encoding': 'chunked',
   'x-amz-bucket-region': 'us-wes

In [138]:
response = requests.get('http://mtgtop8.com/mtgo?d=312531&f=Modern_Valakut_by_philo')

In [142]:
response.content

b"2 Mwonvuli Acid-Moss\r\n1 Hour of Promise\r\n4 Farseek\r\n1 Wood Elves\r\n4 Wooded Foothills\r\n4 Valakut, the Molten Pinnacle\r\n4 Stomping Ground\r\n6 Mountain\r\n2 Misty Rainforest\r\n2 Forest\r\n3 Cinder Glade\r\n1 Bloodstained Mire\r\n2 Prismatic Omen\r\n2 Summoner's Pact\r\n4 Lightning Bolt\r\n2 Sweltering Suns\r\n4 Search for Tomorrow\r\n4 Scapeshift\r\n4 Sakura-Tribe Elder\r\n4 Primeval Titan\r\nSideboard\r\n3 Witchbane Orb\r\n3 Relic of Progenitus\r\n1 Reclamation Sage\r\n2 Nature's Claim\r\n4 Hornet Nest\r\n1 Crumble to Dust\r\n1 Chameleon Colossus\r\n"

In [14]:
import json

In [20]:
initial_list = list()

In [25]:
with open('data/scraped_events.json', 'w') as f:  
    json.dump(initial_list, f)

In [ ]:
json.load()

In [22]:
import boto3

In [23]:
s3 = boto3.client('s3')


In [24]:
s3.get_object(Bucket='mtg-capstone', Key='data/raw_deck_lists/deck_list_303171')

NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [52]:
s3.list_objects_v2?

In [48]:
all_objects = s3.list_objects_v2(Bucket='mtg-capstone', Prefix='data/raw_deck_lists/')

In [49]:
type(all_objects)

dict

In [50]:
list(all_objects.keys())

['ResponseMetadata',
 'IsTruncated',
 'Contents',
 'Name',
 'Prefix',
 'MaxKeys',
 'KeyCount',
 'NextContinuationToken']

In [51]:
all_objects['NextContinuationToken']

'1bTNxfJuDMrW0HuRnUQFsDuTZIGtnWnOAQ5KyoaZshK3UkaUy8EPJgJ0LARPt1cQAXssTnvBGLSVIIz/CjFZhJw=='

In [53]:
s3.list_objects_v2(Bucket='mtg-capstone', Prefix='data/raw_deck_lists/', ContinuationToken=all_objects['NextContinuationToken'])

{'Contents': [{'ETag': '"8e6872d44ecd93778564f4a44f3af181"',
   'Key': 'data/raw_deck_lists/deck_list_302313.txt',
   'LastModified': datetime.datetime(2018, 1, 16, 22, 39, 6, tzinfo=tzutc()),
   'Size': 672,
   'StorageClass': 'STANDARD'},
  {'ETag': '"f64297ede7a797e0664d7d4cf4107778"',
   'Key': 'data/raw_deck_lists/deck_list_302314.txt',
   'LastModified': datetime.datetime(2018, 1, 16, 22, 39, 3, tzinfo=tzutc()),
   'Size': 601,
   'StorageClass': 'STANDARD'},
  {'ETag': '"ca11610802158e2d45c61324c9341853"',
   'Key': 'data/raw_deck_lists/deck_list_302315.txt',
   'LastModified': datetime.datetime(2018, 1, 16, 22, 38, 56, tzinfo=tzutc()),
   'Size': 563,
   'StorageClass': 'STANDARD'},
  {'ETag': '"31477d6d5451c6a5107ef0cd878cf2e6"',
   'Key': 'data/raw_deck_lists/deck_list_302317.txt',
   'LastModified': datetime.datetime(2018, 1, 16, 22, 38, 53, tzinfo=tzutc()),
   'Size': 635,
   'StorageClass': 'STANDARD'},
  {'ETag': '"ea87b002d2604b95a2133236fab44777"',
   'Key': 'data/raw_d

In [ ]:
s3.get_object()